In [1]:
import os
import s3fs
import rasterio as rio
import dask_image
import numpy as np
import dask_image.imread as imread
import dask
import dask.array as da
from dask.distributed import Client, progress

In [2]:
threshold = 2
image_loc = 'slr-interactive/data-test'

In [8]:
# # Create a cluster
# cluster = YarnCluster()

# # Connect to the cluster
# client = Client(cluster)

client = Client(threads_per_worker=1, n_workers=16)
client

Client Scheduler: tcp://127.0.0.1:39825 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 33.72 GB


In [12]:
def process_geotiff(source, threshold):
    with rio.open(source) as src:
        data = src.read() < threshold
        if data.sum() == 0:
            return
        profile = src.profile
        profile['dtype'] = 'uint8'
        profile.pop('nodata')
    
    destination_path = os.path.basename(source)
    with fs.open(os.path.join(image_loc, 'out', destination_path), 'wb') as f:
        with rio.open(f, 'w', nbits=1, **profile) as dst:
            dst.write(data.astype('uint8'))

In [4]:
fs = s3fs.S3FileSystem()
paths = fs.glob(os.path.join(image_loc, '*.tif'))
paths = ['s3://' + path for path in paths]
paths

['s3://slr-interactive/data-test/n36.5w75.8.tif',
 's3://slr-interactive/data-test/n36.5w75.9.tif',
 's3://slr-interactive/data-test/n36.5w76.0.tif',
 's3://slr-interactive/data-test/n36.5w76.1.tif',
 's3://slr-interactive/data-test/n36.5w76.2.tif',
 's3://slr-interactive/data-test/n36.5w76.3.tif',
 's3://slr-interactive/data-test/n36.5w76.4.tif',
 's3://slr-interactive/data-test/n36.5w76.5.tif',
 's3://slr-interactive/data-test/n36.5w76.6.tif',
 's3://slr-interactive/data-test/n36.5w76.7.tif',
 's3://slr-interactive/data-test/n36.5w76.8.tif',
 's3://slr-interactive/data-test/n36.5w76.9.tif',
 's3://slr-interactive/data-test/n36.5w77.0.tif',
 's3://slr-interactive/data-test/n36.5w77.1.tif',
 's3://slr-interactive/data-test/n36.5w77.2.tif',
 's3://slr-interactive/data-test/n36.5w77.3.tif',
 's3://slr-interactive/data-test/n36.5w77.4.tif',
 's3://slr-interactive/data-test/n36.5w77.5.tif',
 's3://slr-interactive/data-test/n36.5w77.6.tif',
 's3://slr-interactive/data-test/n36.5w77.7.tif',


In [9]:
lazy_arrays = [dask.delayed(process_geotiff)(fn, threshold) for fn in paths]

In [10]:
_ = process_geotiff('s3://slr-interactive/data-test/n36.5w79.1.tif', 2)

In [11]:
_ = client.compute(collections=lazy_arrays)